In [1]:
#Load packages
import pandas as pd
import numpy as np
import scipy.io as sio
import random
import scipy
import matplotlib.pyplot as plt
from collections import Counter
from sklearn.manifold import TSNE
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.utils.class_weight import compute_class_weight
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from numpy.linalg import matrix_rank
from scipy.stats import multivariate_t
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC, LinearSVC

In [2]:
#Load data
whole = sio.loadmat('whole_wolabels.mat')
parts = sio.loadmat('parts_wolabels.mat')

In [3]:
#functions

#This function is used to encode labels since labels are categorical.
def encode_labels(labels):
    le = LabelEncoder()
    le.fit(labels)
    encoded_labels = le.transform(labels)
    
    return encoded_labels, le

def decode_labels(encoded_predict_labels, le):
    test_predictions = le.inverse_transform(encoded_predict_labels)
    
    return test_predictions

#Harmonic mean calculation
def Harmonic_Mean(ytest, ypred, ytrain):
    ytest = np.array(ytest)
    ypred = np.array(ypred)
    true_dict = Counter(ytest)
    prediction_dict = Counter(ypred)
    uytest = set(ytest) #all classes present during testing may contain unseen classes and only a subset of the seen classes
    uytrain = set(ytrain) #classes represented during training, seen classes
    
    #Classification f1 score
    uy = list(uytest.intersection(uytrain))
    nc = len(uy)
    Classification_F1 = np.zeros(nc)
    for i in range(nc):
        Index_of_i_th_class_true = list(np.argwhere(ytest == uy[i]).ravel())
        Index_of_i_th_class_pred = list(np.argwhere(ypred == uy[i]).ravel())
        tp = len(list(set(Index_of_i_th_class_true).intersection(Index_of_i_th_class_pred)))
        #print("tp: ", tp)
        fp = prediction_dict[uy[i]] - tp
        #print("fp: ", fp)
        fn = true_dict[uy[i]] - tp
        #print("fn: ", fn)
        Classification_F1[i] = (2*tp)/(2*tp + fp + fn)
    Classification_F1_mean = np.mean(Classification_F1)
    
    #Out of distribution f1 score
    uy = set(uy)
    u_unseen_classes = list(uytest - uy)
    Index_of_unseen_class_true = list(np.argwhere(np.isin(ytest, u_unseen_classes)).ravel())
    Index_of_unseen_class_pred = list(np.argwhere(ypred == -1).ravel())
    tp = len(list(set(Index_of_unseen_class_true).intersection(Index_of_unseen_class_pred)))
    fp = len(Index_of_unseen_class_pred) - tp
    fn = len(Index_of_unseen_class_true) - tp
    OOD_F1 = (2*tp)/(2*tp + fp + fn)
    
    harmonic_mean = 2*Classification_F1_mean*OOD_F1/(Classification_F1_mean+OOD_F1) #harmonic mean of the two scores
    
    return Classification_F1_mean, OOD_F1, harmonic_mean 

In [4]:
#whole

#train
train_classid = np.squeeze(whole['train_classid'])
train_class_labels = []
for item in train_classid:
    train_class_labels.append(item[0])
train_features = whole['train_feats']
train_imid = whole['train_imgid']
train_imgid = []
for item in train_imid:
    train_imgid.append(item[0])
train_imgid = np.squeeze(train_imgid)
train_sampleid = whole['train_sampleid']

#validation
validation_classid = np.squeeze(whole['val_classid'])
validation_class_labels = []
for item in validation_classid:
    validation_class_labels.append(item[0])
validation_features = whole['val_feats']
validation_imid = whole['val_imgid']
validation_imgid = []
for item in validation_imid:
    validation_imgid.append(item[0])
validation_imgid = np.squeeze(validation_imgid)
validation_sampleid = whole['val_sampleid']

#test
test_features = whole['test_feats']
test_imid = whole['test_imgid']
test_imgid = []
for item in test_imid:
    test_imgid.append(item[0])
test_imgid = np.squeeze(test_imgid)
test_sampleid = whole['test_sampleid']

#encoded train labels
train_labels, le = encode_labels(train_class_labels)
train_unique_labels = sorted(np.unique(train_labels))
train_unique_labels_count = len(train_unique_labels)
print(len(train_labels))

#encoded validation labels
validation_labels = le.transform(validation_class_labels)
validation_unique_labels = sorted(np.unique(validation_labels))
validation_unique_labels_count = len(validation_unique_labels)
print(len(validation_labels))

7849
1379


In [5]:
#parts

#train
train_classid_parts = np.squeeze(parts['train_classid'])
train_class_labels_parts = []
for item in train_classid_parts:
    train_class_labels_parts.append(item[0])
train_features_parts = parts['train_feats']
train_imid_parts = parts['train_imgid']
train_imgid_parts = []
for item in train_imid_parts:
    train_imgid_parts.append(item[0])
train_imgid_parts = np.squeeze(train_imgid_parts)
train_sampleid_parts = parts['train_sampleid']
train_tileid_parts = parts['train_tileid']

#validation
validation_classid_parts = np.squeeze(parts['val_classid'])
validation_class_labels_parts = []
for item in validation_classid_parts:
    validation_class_labels_parts.append(item[0])
validation_features_parts = parts['val_feats']
validation_imid_parts = parts['val_imgid']
validation_imgid_parts = []
for item in validation_imid_parts:
    validation_imgid_parts.append(item[0])
validation_imgid_parts = np.squeeze(validation_imgid_parts)
validation_sampleid_parts = parts['val_sampleid']
validation_tileid_parts = parts['val_tileid']

#test
test_features_parts = parts['test_feats']
test_imid_parts = parts['test_imgid']
test_imgid_parts = []
for item in test_imid_parts:
    test_imgid_parts.append(item[0])
test_imgid_parts = np.squeeze(test_imgid_parts)
test_sampleid_parts = parts['test_sampleid']
test_tileid_parts = parts['test_tileid']

combine labels to find outlier percentage

In [6]:
# combine train and validation data labels
total_labels = np.array(list(train_labels) + list(validation_labels))
print(len(total_labels))

9228


Converting into bag representation

In [7]:
train_n, d = train_features_parts.shape
print(train_n, d)
train_features_parts_bags = train_features_parts.reshape(int(train_n/9), 9, d)
print(train_features_parts_bags.shape)

validation_n, d = validation_features_parts.shape
print(validation_n,d)
validation_features_parts_bags = validation_features_parts.reshape(int(validation_n/9), 9, d)
print(validation_features_parts_bags.shape)

70641 384
(7849, 9, 384)
12411 384
(1379, 9, 384)


In [8]:
combined_train_features = []
combined_validation_features = []

for i in range(len(train_features_parts_bags)):
    combined_train_features.append(np.concatenate((train_features[i], train_features_parts_bags[i]), axis=None))
combined_train_features = np.array(combined_train_features)
print(combined_train_features.shape)

for i in range(len(validation_features_parts_bags)):
    combined_validation_features.append(np.concatenate((validation_features[i], validation_features_parts_bags[i]), axis=None))
combined_validation_features = np.array(combined_validation_features)
print(combined_validation_features.shape)

(7849, 3840)
(1379, 3840)


Create holdout dataset for outlier detection

In [9]:
validation_count_dict = Counter(validation_labels)
#print(validation_count_dict)

validation_labels_dict = sorted(validation_count_dict.items(), key=lambda x: x[1], reverse=True)
unique_unseen_labels = []
for i in range(64,114):
      unique_unseen_labels.append(validation_labels_dict[i][0])
print("unique_unseen_labels:", unique_unseen_labels)

unseen_labels_index = np.argwhere(np.isin(total_labels, unique_unseen_labels)).ravel()
print("unseen_labels length:", len(unseen_labels_index))

#separating unseen classes from train data
unseen_labels_index_train = np.argwhere(np.isin(train_labels, unique_unseen_labels)).ravel()
unseen_labels_train = train_labels[unseen_labels_index_train]
unseen_embeddings_train = combined_train_features[unseen_labels_index_train]

#separating seen classes from train data
unique_seen_labels = sorted(list(set(train_unique_labels) - set(unique_unseen_labels)))
print("unique_seen_labels length:", len(unique_seen_labels))

seen_labels_index_train = np.argwhere(np.isin(train_labels, unique_seen_labels)).ravel()
print("seen_labels index train length:", len(seen_labels_index_train))
seen_labels_train = train_labels[seen_labels_index_train].reshape(len(seen_labels_index_train)).tolist()
seen_embeddings_train = combined_train_features[seen_labels_index_train]

#Final train data
train_embeddings = seen_embeddings_train
train_labels = seen_labels_train

#final validation data
validation_embeddings = np.vstack((combined_validation_features, unseen_embeddings_train))
validation_labels = np.concatenate((validation_labels, unseen_labels_train))

unique_unseen_labels: [720, 828, 898, 907, 926, 984, 13, 16, 25, 33, 39, 49, 51, 60, 61, 78, 84, 86, 88, 89, 100, 103, 106, 107, 110, 111, 113, 116, 121, 132, 134, 145, 146, 148, 152, 154, 158, 168, 175, 177, 199, 205, 207, 209, 212, 221, 222, 225, 227, 232]
unseen_labels length: 770
unique_seen_labels length: 963
seen_labels index train length: 7185


verifying shapes

In [10]:
#print(train_embeddings)
print(train_embeddings.shape)
#print(train_labels)
print(len(train_labels))
#print(validation_embeddings)
print(validation_embeddings.shape)
#print(validation_labels)
print(len(validation_labels))

(7185, 3840)
7185
(2043, 3840)
2043


Apply SVM in one-vs-all fashion

SVM classifier with cost 1

In [11]:
predictions = np.zeros(len(validation_labels))
clf = make_pipeline(StandardScaler(), SVC(C=1, kernel = 'linear', class_weight='balanced', random_state=0))
conf_matrix = np.zeros((len(validation_labels), len(unique_seen_labels)))
for j in range(len(unique_seen_labels)):
    positive_labels_index = np.argwhere(np.isin(train_labels, unique_seen_labels[j])).ravel()
    negative_labels_index = list(set(range(len(train_labels))) - set(positive_labels_index))
    positive_labels_embedding = train_embeddings[positive_labels_index]
    negative_labels_embedding = train_embeddings[negative_labels_index]
    positive_labels = unique_seen_labels[j]*np.ones(len(positive_labels_index))
    negative_labels = -1*np.ones(len(negative_labels_index))
    train_embeddings_one = np.vstack((positive_labels_embedding, negative_labels_embedding))
    #print(train_embeddings_one)
    train_labels_one = np.concatenate((positive_labels, negative_labels))
    #print(train_labels_one)
    clf.fit(train_embeddings_one, train_labels_one)
    conf_score = clf.decision_function(validation_embeddings)
    #print(conf_score)
    conf_matrix[:,j] = conf_score

In [12]:
print(conf_matrix.shape)

(2043, 963)


In [13]:
#Mean class accuracy
def mean_class_acc(predictions, true_labels):
    matrix = confusion_matrix(true_labels, predictions)
    acc = matrix.diagonal()/matrix.sum(axis=1)

    return sum(acc)/len(acc)

In [14]:
clf_svm = make_pipeline(StandardScaler(), SVC(C=1, kernel='linear', degree=3, gamma='scale', coef0=0.0, shrinking=True, probability=False, tol=0.001, cache_size=200, class_weight='balanced', verbose=False, max_iter=- 1, decision_function_shape='ovr', break_ties=False, random_state=None))
clf_svm.fit(train_embeddings, train_labels)
preds = clf_svm.predict(validation_embeddings)
score = clf_svm.score(validation_embeddings, validation_labels)
print("Cost parameter:", 1)
print("Overall Accuracy:", score)
mean_acc = mean_class_acc(preds, validation_labels)
print("Mean class accuracy:", mean_acc)

Cost parameter: 1
Overall Accuracy: 0.5139500734214391
Mean class accuracy: 0.7530343628073145


threshold 0

In [15]:
predictions_all = np.zeros(len(validation_embeddings))
for k in range(len(validation_embeddings)):
    if max(conf_matrix[k]) < 0:
        predictions_all[k] = -1
    else:
        predictions_all[k] = unique_seen_labels[np.argmax(conf_matrix[k])]
        
#calculate harmonic mean
Classification_F1_mean, OOD_F1, harmonic_mean = Harmonic_Mean(validation_labels, predictions_all, train_labels)
print("Classification f1 score: ", Classification_F1_mean)
print("OOD f1 score: ", OOD_F1)
print("Harmonic mean: ", harmonic_mean)

Classification f1 score:  0.5812340131736754
OOD f1 score:  0.6865839909808342
Harmonic mean:  0.6295319472525087


In [16]:
predictions_final = preds
for k in range(len(validation_embeddings)):
    if max(conf_matrix[k]) < 0:
        predictions_final[k] = -1
        
print(list(predictions_final).count(-1))

#calculate harmonic mean
Classification_F1_mean, OOD_F1, harmonic_mean = Harmonic_Mean(validation_labels, predictions_final, train_labels)
print("Classification f1 score: ", Classification_F1_mean)
print("OOD f1 score: ", OOD_F1)
print("Harmonic mean: ", harmonic_mean)

1004
Classification f1 score:  0.5806923257161002
OOD f1 score:  0.6865839909808342
Harmonic mean:  0.6292140857824445


threshold -0.1

In [ ]:
predictions = np.zeros(len(validation_embeddings))
for k in range(len(validation_embeddings)):
    if max(conf_matrix[k]) < -0.1:
        predictions[k] = -1
    else:
        predictions[k] = unique_seen_labels[np.argmax(conf_matrix[k])]
        
#calculate harmonic mean
Classification_F1_mean, OOD_F1, harmonic_mean = Harmonic_Mean(validation_labels, predictions, train_labels)
print("Classification f1 score: ", Classification_F1_mean)
print("OOD f1 score: ", OOD_F1)
print("Harmonic mean: ", harmonic_mean)

In [ ]:
predictions_final = preds
for k in range(len(validation_embeddings)):
    if max(conf_matrix[k]) < -0.1:
        predictions_final[k] = -1
        
print(list(predictions_final).count(-1))

#calculate harmonic mean
Classification_F1_mean, OOD_F1, harmonic_mean = Harmonic_Mean(validation_labels, predictions_final, train_labels)
print("Classification f1 score: ", Classification_F1_mean)
print("OOD f1 score: ", OOD_F1)
print("Harmonic mean: ", harmonic_mean)

species prediction from LR_liblinear

In [20]:
LR_pred = pd.read_csv('Task3_preds_numerical_LR_c10_liblinear_normalized.csv', header = None)[0]
print(LR_pred)

0         0
1         1
2         2
3         3
4         4
       ... 
2038    988
2039    988
2040    988
2041    985
2042    988
Name: 0, Length: 2043, dtype: int64


In [21]:
predictions_final = LR_pred
for k in range(len(validation_embeddings)):
    if max(conf_matrix[k]) < -0.3:
        predictions_final[k] = -1
        
print(list(predictions_final).count(-1))

#calculate harmonic mean
Classification_F1_mean, OOD_F1, harmonic_mean = Harmonic_Mean(validation_labels, predictions_final, train_labels)
print("Classification f1 score: ", Classification_F1_mean)
print("OOD f1 score: ", OOD_F1)
print("Harmonic mean: ", harmonic_mean)

616
Classification f1 score:  0.7120638079482344
OOD f1 score:  0.6435786435786436
Harmonic mean:  0.6760913383091742


In [ ]:
645
Classification f1 score:  0.6978182356408296
OOD f1 score:  0.6600706713780918
Harmonic mean:  0.6784197866532826

In [ ]:
#print(conf_matrix)
for k in range(len(validation_labels)):
    if max(conf_matrix[k]) < 0:
        predictions[k] = -1
    else:
        predictions[k] = unique_seen_labels[np.argmax(conf_matrix[k])]

        #calculate harmonic mean
Classification_F1_mean, OOD_F1, harmonic_mean = Harmonic_Mean(validation_labels, predictions, train_labels)
print("Classification f1 score: ", Classification_F1_mean)
print("OOD f1 score: ", OOD_F1)
print("Harmonic mean: ", harmonic_mean)